In [17]:
include("LiPoSID.jl")
using LinearAlgebra

Augmented Bloch-4 Vector:

$
\begin{pmatrix} b_x\\
b_y \\
b_z \\
1
\end{pmatrix} = 
\begin{pmatrix} \rho_{10} + \rho_{01}\\
i(\rho_{01} - \rho_{10}) \\
\rho_{00} - \rho_{11} \\
1 
\end{pmatrix} = 
\begin{pmatrix} 2 \operatorname{Re}(\rho_{01})\\
2 \operatorname{Im}(\rho_{10}) \\
\rho_{00} - \rho_{11} \\
\rho_{00} + \rho_{11} 
\end{pmatrix}
$

In [18]:
function bloch4(ρ)

    b = convert.(Float64, [ ρ[1,2] + ρ[2,1],
                           (ρ[1,2] - ρ[2,1])*im,
                            ρ[1,1] - ρ[2,2],
                            ρ[1,1] + ρ[2,2]       ])

end

bloch4 (generic function with 1 method)

Density matrix represented with Bloch vector components:

$\rho = \frac{1}{2}\begin{pmatrix} 1+ b_z& b_x-i b_y \\ b_x + i b_y  & 1-b_z
   \end{pmatrix}$

In [19]:
function dm_b4(b) 

    ρ = [ 1+b[3]         b[2]-im*b[3]
          b[2]+im*b[3]   1-b[3]       ]/2

end

dm_b4 (generic function with 1 method)

In [20]:
γ = 0.079477 
ω = 25.126

25.126

Linear operator that corresponds to Lindblad master equation with one dissipator:

$
\begin{pmatrix} \dot{b_x} \\
\dot{b_y} \\
\dot{b_z} \\
0
\end{pmatrix} = 
\begin{pmatrix} -\gamma/2 & -\omega & 0 & 0\\
\omega & -\gamma/2 & 0 & 0\\
0 & 0 & -\gamma & \gamma \\
0 & 0 &  0 & 0
\end{pmatrix}
\begin{pmatrix} b_x\\
b_y \\
b_z \\
1
\end{pmatrix} 
$

In [21]:
Aᴸᴹᴱ  = [ -γ/2  -ω     0   0
             ω  -γ/2   0   0
             0   0    -γ   γ 
             0   0     0   0 ] 

4×4 Matrix{Float64}:
 -0.0397385  -25.126       0.0       0.0
 25.126       -0.0397385   0.0       0.0
  0.0          0.0        -0.079477  0.079477
  0.0          0.0         0.0       0.0

Linear operator identified from Kurt spin-boson data with DMD4

$
\begin{pmatrix} \dot{b_x} \\
\dot{b_y} \\
\dot{b_z} \\
0
\end{pmatrix} = 
\begin{pmatrix} 0 & -\omega & 0 & 0\\
\omega & -\gamma & 0 & 0\\
0 & 0 & -\gamma & \gamma \\
0 & 0 &  0 & 0
\end{pmatrix}
\begin{pmatrix} b_x\\
b_y \\
b_z \\
1
\end{pmatrix} 
$

In [22]:
Aˢᵇ  =   [  0   -ω     0   0
            ω   -γ     0   0
            0    0    -γ   γ 
            0    0     0   0]  


#eigen(Aˢᵇ)
#M = eigvecs(exp(10*Aˢᵇ))

4×4 Matrix{Float64}:
  0.0    -25.126      0.0       0.0
 25.126   -0.079477   0.0       0.0
  0.0      0.0       -0.079477  0.079477
  0.0      0.0        0.0       0.0

$
\rho^{(g)}(0) =  \rho^{\ket{0}}(0) = \left(
\begin{array}{c}
            1  \\
            0   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1 & 0
                \end{array}
                \right) = 
\ket{0} \bra{0} = \left(
        \begin{array}{ccccc}
            1  &  0  \\
            0  &  0   \\
        \end{array}
            \right), \notag \\
\rho^{(e)}(0)=\rho^{\ket{1}}(0)=\ket{1} \bra{1} = \left(
        \begin{array}{ccccc}
            0  &  0  \\
            0  &  1  \\
        \end{array}
            \right), \notag\\
\rho^{\ket{x}}(0) =\ket{x} \bra{x} = |+\rangle \langle + | = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  1  \\
            1  &  1   \\
        \end{array}
    \right), \notag\\
\rho^{\ket{y}}(0)  = \ket{y} \bra{y} = |-\rangle \langle-| = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  -i  \\
            i  &  1   \\
        \end{array}
    \right)$

In [23]:
ψ⁰ = [1   0.0+0im];  ρᵍ₀ = ψ⁰'ψ⁰ ; bᵍ₀ = LiPoSID.bloch(ρᵍ₀)
ψ¹ = [0   1.0+0im];  ρᵉ₀ = ψ¹'ψ¹ ; bᵉ₀ = LiPoSID.bloch(ρᵉ₀)

3-element Vector{Float64}:
  0.0
  0.0
 -1.0

In [24]:
ρˣ₀ = [ 1. 1
        1  1.0+0im ]; bˣ₀ = LiPoSID.bloch(ρˣ₀)

ρʸ₀ = [ 1.  -im
        im   1.0+0im  ]; bʸ₀ = LiPoSID.bloch(ρʸ₀)

3-element Vector{Float64}:
 0.0
 2.0
 0.0

In [25]:
function 𝓔(ρ₀, Ac)
    Δt = 0.02
    A = exp(Ac * Δt)
    ρᶠ = dm_b4(A * bloch4(ρ₀))
    return ρᶠ
end

𝓔 (generic function with 1 method)

$\rho_1' = \mathcal{E}(|0\rangle \langle 0 |)$

$\rho_4' = \mathcal{E}(|1\rangle \langle 1 |)$

$\rho_2' = \mathcal{E}(|+\rangle \langle + |) -i  \mathcal{E}(|-\rangle \langle-|) - (1\textbf{+}i) (\rho_1' + \rho_4')/2, \quad$ - it is $(1\textbf{-}i)$ in Nielsen&Chuang which is WRONG!

$\rho_3' = \mathcal{E}(|+\rangle \langle + |) +i  \mathcal{E}(|-\rangle \langle-|) - (1+i) (\rho_1' + \rho_4')/2$

In [147]:
function QPT(Ac)

    ρ₁ = 𝓔(ρᵍ₀, Ac)
    ρ₄ = 𝓔(ρᵉ₀, Ac)
    ρ₂ = 𝓔(ρˣ₀, Ac) + im*𝓔(ρʸ₀, Ac) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    ρ₃ = 𝓔(ρˣ₀, Ac) - im*𝓔(ρʸ₀, Ac) - (1+im)*(ρ₁+ρ₄)/2 

    #ρ₂ = 𝓔(ρˣ₀, Ac) - im*𝓔(ρʸ₀, Ac) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    #ρ₃ = 𝓔(ρˣ₀, Ac) + im*𝓔(ρʸ₀, Ac) - (1+im)*(ρ₁+ρ₄)/2 

    σₓ = [ 0  1
           1  0 ]  # X gate

    Λ = [ I  σₓ
          σₓ  -I ] / 2

    #Ρ = [ρ₁ ρ₃ 
    #     ρ₂ ρ₄] 

    Ρ = [ρ₁ ρ₂
         ρ₃ ρ₄] 
     
    χ = Λ*Ρ*Λ

    return χ 
end

χˢᵇ = QPT(Aˢᵇ)
χᴸᴹᴱ = QPT(Aᴸᴹᴱ)
dᴸᴹᴱ, Uᴸᴹᴱ = eigen(χᴸᴹᴱ)
dˢᵇ, Uˢᵇ = eigen(χˢᵇ)
Uᴸᴹᴱ * diagm(dᴸᴹᴱ) * Uᴸᴹᴱ' ≈ χᴸᴹᴱ
Uˢᵇ * diagm(dˢᵇ) * Uˢᵇ' ≈ χˢᵇ

true

In [148]:
E₀ = [1.  0
      0   1 ]
E₁ = [ 0.  1
       1  0 ]  #  σₓ  or X gate
E₂ = -im * [ 0  -im
             im  0  ]
E₃ = [ 1  0
       0 -1 ]
Ẽ = [E₀, E₁, E₂, E₃] 

d, U = eigen(χˢᵇ)

Ẽ[1]
sum(U[j,1] * Ẽ[j] for j in 1:size(U)[i])


2×2 Matrix{ComplexF64}:
 0.200965+0.541299im  -0.736255+0.473275im
 0.665096+0.473275im  -0.477494+0.0787711im

In [149]:
function operator_sum(χ)
    d, U = eigen(χ)
    #@assert U * diagm(d) * U' ≈ χ
    E = []
    for i in 1:size(U)[2]
        push!(E, sqrt(d[i] * sum(U[j,i] * Ẽ[j] for j in 1:size(U)[1])))
    end
    return E
end

operator_sum (generic function with 1 method)

In [150]:
Eˢᵇ = operator_sum(χˢᵇ)
Eˢᵇ[3]

2×2 Matrix{ComplexF64}:
   0.41133+0.512653im   0.316661-0.690682im
 0.0784934-0.440147im  -0.235292-0.0088777im

In [151]:
Eᴸᴹᴱ = operator_sum(χᴸᴹᴱ)
Eᴸᴹᴱ[3]

2×2 Matrix{ComplexF64}:
  0.410714+0.512122im   0.315323-0.690558im
 0.0783229-0.43987im   -0.235115-0.00785649im

In [152]:
Eˢᵇ[4]

2×2 Matrix{ComplexF64}:
 0.956578+0.270644im  0.323712-0.0402712im
 0.117543+0.369427im   1.03078-0.362586im

In [153]:
Eᴸᴹᴱ[4]

2×2 Matrix{ComplexF64}:
 0.956688+0.27074im   0.323812-0.0403083im
  0.11757+0.369462im   1.03087-0.362714im

In [154]:
sum(Eᴸᴹᴱ)


2×2 Matrix{ComplexF64}:
   1.86487+0.437817im  1.19441-0.905679im
 -0.185553-0.435982im  1.58895-0.316759im

In [155]:
sum(Eˢᵇ)

2×2 Matrix{ComplexF64}:
   1.86061+0.438704im  1.19279-0.907315im
 -0.181642-0.439338im  1.59086-0.318407im